In [ ]:
#-------------------------------------------------------------
#-------------------------------------------------------------
#-----------------CARGA DEL MODELO WHISPER--------------------
#-------------------------------------------------------------
#-------------------------------------------------------------
#-------------------------------------------------------------

import torch, whisper

print("CUDA disponible:", torch.cuda.is_available())

# esto volverá a descargar medium de cero
model = whisper.load_model("medium", device="cpu")

# luego lo pasas a GPU en float16
model = model.to(torch.device("cuda"), dtype=torch.float16)

print("Modelo medium listo en", next(model.parameters()).dtype)


CUDA disponible: True


100%|█████████████████████████████████████| 1.42G/1.42G [04:17<00:00, 5.93MiB/s]


Modelo medium listo en torch.float16


In [ ]:
#-------------------------------------------------------------
#-------------------------------------------------------------
#------------Primera prueba, Transcribía lento----------------
#-------------------------------------------------------------
#-------------------------------------------------------------
#-------------------------------------------------------------

import sounddevice as sd
from scipy.io.wavfile import write
import whisper, torch, tempfile

SAMPLERATE = 16000  # Hz
DURATION   = 5      # segundos

# 1) Cargar Whisper medium directamente en GPU (float32)
print("Cargando Whisper medium en GPU…")
model = whisper.load_model("medium", device="cuda")
print("Modelo cargado en GPU con dtype:", next(model.parameters()).dtype)

def grabar_y_transcribir():
    print(f"Grabando {DURATION}s de audio…")
    audio = sd.rec(int(SAMPLERATE * DURATION),
    samplerate=SAMPLERATE, channels=1,dtype='int16')
    sd.wait()
    print("Grabación completada.")

    # Guardar en WAV temporal
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        write(f.name, SAMPLERATE, audio)
        print("Transcribiendo (español, fp32)…")
        result = model.transcribe(
            f.name,
            language="es",   # omite detect_language
            fp16=False       # fuerza float32 en el decodificador
        )
        print("Texto reconocido:", result["text"])

if __name__ == "__main__":
    grabar_y_transcribir()


Cargando Whisper medium en GPU…
Modelo cargado en GPU con dtype: torch.float32
Grabando 5s de audio…
Grabación completada.
Transcribiendo (español, fp32)…
Texto reconocido:  Hola que tal, esta es la primera prueba


In [ ]:
'''grabar_y_transcribir()'''

Grabando 5s de audio…
Grabación completada.
Transcribiendo (español, fp32)…
Texto reconocido:  si yo quiero decir por ejemplo junta 1 30 grados a la derecha


In [ ]:
#-----------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------
#---Segunda prueba, Aquí ya se hizo una demo de un sistema de reconocimiento que se activa al decir "hola"--------------------
#-----------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------

import sounddevice as sd
import numpy as np
import queue, threading, time, tempfile, os, re, difflib, collections
import whisper, torch, unidecode, webrtcvad
from scipy.io.wavfile import write

# ─── Parámetros globales ──────────────────────────────────────────────────────
SAMPLERATE      = 16_000    # Hz
BLOCK_SEC       = 2         # segundos por bloque de audio
KWS_NBLK        = 2         # Nº de bloques concatenados para KWS (mejor contexto)
TRIGGER_PHRASE  = "hola"    # frase de activación (sin tildes)
TRIGGER_SIM_TH  = 0.85      # umbral de similitud para activar
MAX_CMD_TIME    = 3         # s máximos para comando
SILENCE_SEC     = 1         # parar si silencio prolongado ≥ 1 s
SILENCE_LVL     = 0.05      # umbral RMS para “silencio”

# ---- WebRTC‑VAD frame -----------------
VAD_FRAME_MS    = 30
VAD_FRAME_SAMP  = int(SAMPLERATE * VAD_FRAME_MS / 1000)   # 480 a 16 kHz
# ---------------------------------------

# ─── Cargar modelos Whisper ──────────────────────────────────────────────────
print("Cargando modelo *small* para keyword‑spotting …")
model_kws = whisper.load_model("small", device="cuda")

print("Cargando modelo *medium* para comando …")
model_cmd = whisper.load_model("medium", device="cuda")

# ─── Simulador de transmisión serial ────────────────────────────────────────
def send_serial_sim(msg: str):
    """Imita la escritura a un puerto serie mostrando la cadena en pantalla."""
    print(f"[SERIAL] {msg}")

# ─── VAD basado en WebRTC ───────────────────────────────────────────────────
vad = webrtcvad.Vad(2)   # agresividad 0–3

def is_speech(block_f32: np.ndarray) -> bool:
    """True si alguna sub‑trama de 30 ms contiene voz."""
    pcm = (block_f32 * 32767).astype(np.int16)
    bytes_pcm = pcm.tobytes()
    for i in range(0, len(pcm), VAD_FRAME_SAMP):
        frame = bytes_pcm[i*2 : (i+VAD_FRAME_SAMP)*2]
        if len(frame) < VAD_FRAME_SAMP * 2:
            break
        if vad.is_speech(frame, SAMPLERATE):
            return True
    return False

# ─── Colas y estado ─────────────────────────────────────────────────────────
audio_q = queue.Queue()
state   = "idle"            # "idle" | "waiting_cmd"
cmd_start_time = 0
cmd_buffer: list[np.ndarray] = []
kws_buffer = collections.deque(maxlen=KWS_NBLK)  # ventana deslizante para KWS

# ─── Captura de audio (hilo productor) ──────────────────────────────────────
def audio_callback(indata, frames, time_info, status):
    audio_q.put(indata.copy())

def producer():
    with sd.InputStream(samplerate=SAMPLERATE,
                        channels=1,
                        blocksize=int(SAMPLERATE*BLOCK_SEC),
                        dtype="float32",
                        callback=audio_callback):
        print("» Productor de audio activo.")
        while True:
            time.sleep(0.1)

# ─── Transcripción con Whisper ──────────────────────────────────────────────
def whisper_text(model, wav_f32):
    tmp = tempfile.mktemp(suffix=".wav")
    write(tmp, SAMPLERATE, (wav_f32*32767).astype(np.int16))
    out = model.transcribe(tmp, language="es", fp16=False)["text"].lower().strip()
    os.unlink(tmp)
    return out

# ─── Función helper: similitud con la frase clave ──────────────────────────
def is_trigger(text):
    return difflib.SequenceMatcher(None, text, TRIGGER_PHRASE).ratio() >= TRIGGER_SIM_TH

# ─── Procesar y “enviar” comando ────────────────────────────────────────────
def process_sentence(text):
    clean = unidecode.unidecode(text)
    m = re.search(r"junta\s+(\d+).*?(-?\d+)\s*grados", clean)
    if m:
        joint, angle = int(m.group(1)), int(m.group(2))
        print(f"→ Orden decodificada: J{joint}={angle}")
        send_serial_sim(f"J{joint}:{angle}")
    else:
        print("   (No se entendió ningún comando válido)")

# ─── Máquina de estados (hilo consumidor) ──────────────────────────────────
def consumer():
    global state, cmd_start_time, cmd_buffer
    silent_blocks = 0
    print("» Consumidor activo. Esperando la frase clave…")

    while True:
        block = audio_q.get().flatten()

        # ---------- Estado: IDLE ----------
        if state == "idle":
            if np.sqrt(np.mean(block**2)) < SILENCE_LVL or not is_speech(block):
                continue
            kws_buffer.append(block)
            if len(kws_buffer) < KWS_NBLK:
                continue
            wav_kws = np.concatenate(kws_buffer, axis=0)
            text_kws = whisper_text(model_kws, wav_kws)
            print("(kws)", text_kws)
            if is_trigger(text_kws):
                print("» Frase clave detectada → grabando comando…")
                state = "waiting_cmd"
                cmd_start_time = time.time()
                cmd_buffer = []
                silent_blocks = 0

        # ---------- Estado: WAITING_CMD ----------
        elif state == "waiting_cmd":
            cmd_buffer.append(block)
            if np.sqrt(np.mean(block**2)) < SILENCE_LVL or not is_speech(block):
                silent_blocks += 1
            else:
                silent_blocks = 0

            if time.time() - cmd_start_time >= MAX_CMD_TIME or silent_blocks * BLOCK_SEC >= SILENCE_SEC:
                wav_cmd = np.concatenate(cmd_buffer, axis=0)
                text_cmd = whisper_text(model_cmd, wav_cmd)
                print("» Comando detectado →", text_cmd)
                process_sentence(text_cmd)
                state = "idle"
                kws_buffer.clear()

# ─── Main ───────────────────────────────────────────────────────────────────
def main():
    threading.Thread(target=producer, daemon=True).start()
    consumer()

if __name__ == "__main__":
    main()



Cargando modelo *small* para keyword‑spotting …
Cargando modelo *medium* para comando …
» Consumidor activo. Esperando la frase clave…
» Productor de audio activo.
(kws) hola. hola.
(kws) hola. hola.
(kws) hola. hola.
(kws) hola. hola.
(kws) hola. hola.
(kws) hola. hola.
(kws) hola.
» Frase clave detectada → grabando comando…
» Comando detectado → hola hola
   (No se entendió ningún comando válido)
(kws) no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no
(kws) 
(kws) junta, junta, uno, tres.
(kws) junta, uno, treinta junta, uno, treinta
(kws) junta, 1-30 junta, 1-30
(kws) junta 130 grados
(kws) 1 

KeyboardInterrupt: 